<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/RSI_Cross_UP_SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Google Colab için Gerekli Kütüphaneleri Yükleme
!pip install tradingview_screener
!pip install git+https://github.com/rongardF/tvdatafeed matplotlib openpyxl
!pip install tradingview-screener==2.5.0
!pip install tqdm
!pip install ta

# 📌 Gerekli Kütüphaneler
try:
    import tradingview_screener
    from tradingview_screener import get_all_symbols
    import pandas as pd
    import numpy as np
    from tvDatafeed import TvDatafeed, Interval
    from tqdm import tqdm
    from datetime import datetime
    import warnings
    import ta
    from tabulate import tabulate
    # 📌 Uyarıları Engelle
    warnings.filterwarnings("ignore", category=FutureWarning)

except ImportError:
    print("Eksik kütüphaneler var, lütfen kurulumları tamamlayın.")
    exit()

# 📌 Kullanıcıdan Zaman Dilimi Seçimi
zaman_periyotlari = {
    "1": ("1 Saatlik", Interval.in_1_hour, 100),  # 1 saatlik için 100 bar
    "4": ("4 Saatlik", Interval.in_4_hour, 200),  # 4 saatlik için 200 bar
    "D": ("Günlük", Interval.in_daily, 400),      # Günlük için 400 bar
    "W": ("Haftalık", Interval.in_weekly, 150),   # Haftalık için 150 bar
    "M": ("Aylık", Interval.in_monthly, 55),      # Aylık için 55 bar
}

zaman_dilimleri_ad = {
    "1": "1 Saatlik",
    "4": "4 Saatlik",
    "D": "Günlük",
    "W": "Haftalık",
    "M": "Aylık",
}

print("\n📌 Lütfen analiz etmek istediğiniz zaman dilimini seçin:")
for key, val in zaman_dilimleri_ad.items():
    print(f"{key} - {val}")

zaman_secimi = input("Seçiminizi yapın (1, 4, D, W, M): ").upper()

if zaman_secimi not in zaman_periyotlari:
    print("⚠ Geçersiz seçim! Varsayılan olarak Günlük analiz yapılıyor.")
    zaman_secimi = "D"

interval_label, interval, n_bars = zaman_periyotlari[zaman_secimi]

print(f"📌 Seçilen Zaman Periyodu: {interval_label}, Mum Sayısı: {n_bars}")

# 📌 Kullanıcıdan RSI Aralığı Seçimi
rsi_araliklari = {
    "1": (0, 20),
    "2": (20, 40),
    "3": (40, 50),
    "4": (40, 60),
    "5": (50, 70),
}

print("\n📌 RSI seviye aralığını seçin:")
for key, val in rsi_araliklari.items():
    print(f"{key} - {val[0]} - {val[1]}")

rsi_secimi = input("Seçiminizi yapın (1, 2, 3, 4, 5): ")

if rsi_secimi not in rsi_araliklari:
    print("⚠ Geçersiz seçim! Varsayılan olarak 30'u geçen hisseler aranacak.")
    rsi_aralik = (30, 100)  # Varsayılan olarak RSI 30 üzeri olanları alırız
else:
    rsi_aralik = rsi_araliklari[rsi_secimi]

print(f"📌 Seçilen RSI Aralığı: {rsi_aralik[0]} - {rsi_aralik[1]}")

# 📌 TradingView Bağlantısı
tv = TvDatafeed()

# 📌 BIST Hisselerini Al
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# 📌 Tarama Sonuçları DataFrame
tarama_sonuclari = pd.DataFrame(columns=['Hisse', 'RSI (14)', 'RSI - SMA Durumu', 'Son Fiyat'])

# 📌 Tarama İşlemi
for hisse in tqdm(Hisseler, desc=f"📊 {interval_label} RSI {rsi_aralik[0]} - {rsi_aralik[1]} aralığında taranıyor", unit="hisse"):
    try:
        df = tv.get_hist(hisse, exchange='BIST', interval=interval, n_bars=n_bars)

        if df is None or len(df) < 14:
            print(f"⚠ {hisse}: Yeterli veri yok, taramaya dahil edilmedi.")
            continue

        # 📌 RSI Hesaplama
        df["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()

        # 📌 RSI'nın SMA'sını hesapla (14 periyotluk SMA)
        df["RSI_SMA"] = df["RSI"].rolling(window=14).mean()

        # 📌 Önceki ve Güncel RSI Değerleri
        anlik_rsi = df["RSI"].iloc[-2]  # Önceki günün RSI değeri
        son_rsi = df["RSI"].iloc[-1]    # Güncel RSI değeri

        # 📌 Önceki ve Güncel RSI SMA Değerleri
        anlik_rsi_sma = df["RSI_SMA"].iloc[-2]  # Önceki gün RSI SMA değeri
        son_rsi_sma = df["RSI_SMA"].iloc[-1]    # Güncel RSI SMA değeri

        # 📌 Ekrana gerçek RSI değerlerini yazdır
        print(f"{hisse}: Önceki RSI = {anlik_rsi}, Güncel RSI = {son_rsi}, Seviye Aralığı = {rsi_aralik}")

        # 📌 1. Adım: RSI'nın Belirli Aralıkta Olup Olmadığını Kontrol Et
        if rsi_aralik[0] <= son_rsi <= rsi_aralik[1]:
            print(f"{hisse} RSI {rsi_aralik} aralığında! 🚀")

            # 📌 2. Adım: RSI - SMA İlişkisini Kontrol Et
            if son_rsi > son_rsi_sma:
                if anlik_rsi < anlik_rsi_sma:
                    rsi_sma_durumu = "RSI SMA'yı Yukarı Kesti ✅"
                else:
                    rsi_sma_durumu = "RSI SMA'nın Üstünde"
            else:
                if anlik_rsi > anlik_rsi_sma:
                    rsi_sma_durumu = "RSI SMA'yı Aşağı Kesti ❌"
                else:
                    rsi_sma_durumu = "RSI SMA'nın Altında"

            # 📌 3. Adım: RSI Yönü Yatay Durumu Ekleme
            rsi_degisim = abs(son_rsi - anlik_rsi)  # RSI değişim miktarı
            if rsi_degisim < 1:  # %1'den az değişim yatay kabul edilir
                rsi_sma_durumu += " | RSI Yönü Yatay ➡️"

            # 📌 4. Adım: Sonuçları DataFrame'e Ekle
            tarama_sonuclari.loc[len(tarama_sonuclari)] = [
                hisse, round(son_rsi, 2), rsi_sma_durumu, round(df["close"].iloc[-1], 2)
            ]

    except Exception as e:
        print(f"{hisse} için hata: {e}")

# 📌 1. Liste: RSI Aralığına Uyan Tüm Hisseler
print("\n📊 **RSI Sonuçları (SMA Bilgisi ile)**")
print(tabulate(tarama_sonuclari, headers="keys", tablefmt="grid"))

# 📌 2. Liste: RSI SMA'yı Yukarı Kesten veya RSI SMA'nın Üzerinde & RSI Yönü Yukarı Olan Hisseler
yukari_kesen_veya_ustunde_hisseler = tarama_sonuclari[
    (tarama_sonuclari["RSI - SMA Durumu"].str.contains("Yukarı Kesti")) |
    (
        (tarama_sonuclari["RSI - SMA Durumu"].str.contains("RSI SMA'nın Üstünde")) &
        (~tarama_sonuclari["RSI - SMA Durumu"].str.contains("Yatay"))
    )
]

print("\n🚀 **RSI SMA'yı Yukarı Kesten veya RSI SMA'nın Üzerinde & RSI Yönü Yukarı Olan Hisseler**")
print(tabulate(yukari_kesen_veya_ustunde_hisseler, headers="keys", tablefmt="grid"))


# 📌 Tarama Sonucu Bilgilendirmesi
tarih_saat = datetime.now().strftime("%Y-%m-%d %H:%M")
toplam_hisse = len(tarama_sonuclari)
yukari_kesen_sayi = len(yukari_kesen_veya_ustunde_hisseler)  # Düzeltme burada


print(f"""
📊 **{interval_label} RSI {rsi_aralik[0]} - {rsi_aralik[1]} Aralığında Hisseler (RSI - SMA Bilgili)**
{tarih_saat} tarihinde yapılan tarama sonucunda, {toplam_hisse} adet hisse bulundu.
🚀 **RSI SMA'yı Yukarı Kesten Hisseler**: {yukari_kesen_sayi} adet
""")

# 📌 Sonuçları Excel'e Kaydet
output_file = f"RSI_SMA_ANALIZ_{interval_label}_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
tarama_sonuclari.to_excel(output_file, index=False)
print(f"\n💾 **Sonuçlar '{output_file}' dosyasına kaydedildi.** ✅")




  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-bbxwvo4k
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-bbxwvo4k
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=864a2d3ff609c2383dd0e0002754aaf71a4664d86982a28932da53a44ff70e13
  Stored in directory: /tmp/pip-ephem-wheel-cache-39mxxrwr/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: tradingview-screener
    Found existing installation: tradingview-screener 3.0.0
    Uninstalling tradingview-screener-3.0.0:
      Successfully uninstalled tradingview-screener-3.0.0
  Preparing metadata (setup.py) ... done
  Created wheel

📌 Seçilen RSI Aralığı: 40 - 50


📊 Haftalık RSI 40 - 50 aralığında taranıyor:   0%|          | 1/601 [00:00<06:12,  1.61hisse/s]

A1CAP: Önceki RSI = 56.93835206633298, Güncel RSI = 41.79496551048105, Seviye Aralığı = (40, 50)
A1CAP RSI (40, 50) aralığında! 🚀


📊 Haftalık RSI 40 - 50 aralığında taranıyor:   0%|          | 2/601 [00:01<06:01,  1.66hisse/s]

...

📊 Haftalık RSI 40 - 50 aralığında taranıyor: 100%|█████████▉| 600/601 [05:31<00:00,  2.07hisse/s]

ZRGYO: Önceki RSI = 88.68383896317847, Güncel RSI = 75.93291542529214, Seviye Aralığı = (40, 50)


📊 Haftalık RSI 40 - 50 aralığında taranıyor: 100%|██████████| 601/601 [05:32<00:00,  1.81hisse/s]

ZSR25: Önceki RSI = 63.84472925906463, Güncel RSI = 38.541602660356034, Seviye Aralığı = (40, 50)

📊 **RSI Sonuçları (SMA Bilgisi ile)**
+-----+---------+------------+-----------------------------------------+-------------+
|     | Hisse   |   RSI (14) | RSI - SMA Durumu                        |   Son Fiyat |
+=====+=========+============+=========================================+=============+
|   0 | A1CAP   |      41.79 | RSI SMA'yı Aşağı Kesti ❌               |        4.48 |
+-----+---------+------------+-----------------------------------------+-------------+
|   1 | ADGYO   |      41.33 | RSI SMA'yı Aşağı Kesti ❌               |       27.98 |
+-----+---------+------------+-----------------------------------------+-------------+
|   2 | AFYON   |      43.66 | RSI SMA'yı Aşağı Kesti ❌               |       13.68 |
+-----+---------+------------+-----------------------------------------+-------------+
|   3 | AGHOL   |      40.09 | RSI SMA'yı Aşağı Kesti ❌               |      276.75


💾 **Sonuçlar 'RSI_SMA_ANALIZ_Haftalık_20250322_0700.xlsx' dosyasına kaydedildi.** ✅
